# Documentación: Versión 2 - Lighthouse Assistant
---

## 1. Resumen de Cambios

### Nuevas Funcionalidades Implementadas

Se han implementado tres grandes mejoras al sistema:

1. **Sistema de Carga de Reportes**: Los usuarios pueden subir múltiples reportes JSON de Google Lighthouse
2. **Preprocesamiento Inteligente**: Reducción automática del tamaño de reportes (20-95%)
3. **Resumen con LLM**: Conversión de JSON a resúmenes en lenguaje natural usando un modelo secundario
4. **Arquitectura Multi-Modelo**: Uso de dos modelos LLaMA para optimizar costos y calidad
5. **Control de temperatura**: El usuario puede ajustar la creatividad del resumen generado

### Beneficios

- ✅ **Reducción drástica de tokens**: De cientos de miles a menos de 5000
- ✅ **Optimización de costos**: Usa modelo pequeño (8b-instant) para tareas de resumen
- ✅ **Mejor calidad**: El modelo grande recibe información pre-procesada
- ✅ **Escalabilidad**: Maneja reportes de cualquier tamaño
- ✅ **Experiencia mejorada**: Feedback visual en tiempo real

## 2. Sistema de Carga de Reportes JSON

### `ui/layout.py` - Modificaciones

Se agregó funcionalidad completa para cargar, procesar y gestionar reportes JSON de Google Lighthouse.

In [ ]:
import streamlit as st
import json


def render_layout():
    """
    Renderiza el layout con funcionalidad de carga de reportes.
    
    Nuevas características:
    - Sube múltiples archivos JSON
    - Valida formato JSON
    - Almacena en session_state
    - Muestra lista de reportes cargados
    - Permite eliminar reportes individuales
    """
    
    st.set_page_config(page_title="Google Lighthouse Assistant", layout="wide")
    st.title("Asistente :blue[Google Lighthouse]")
    
    with st.sidebar:
        st.subheader("Reportes Lighthouse")
        
        # 1. File uploader con soporte para múltiples archivos
        uploaded_files = st.file_uploader(
            "Carga tus reportes JSON",
            type="json",
            accept_multiple_files=True  # ← NUEVO: múltiples archivos
        )
        
        # 2. Procesar archivos subidos
        if uploaded_files:
            if "lighthouse_reports" not in st.session_state:
                st.session_state.lighthouse_reports = {}
            
            for uploaded_file in uploaded_files:
                try:
                    # Leer y parsear JSON
                    report_data = json.load(uploaded_file)
                    file_name = uploaded_file.name
                    
                    # Almacenar si es nuevo
                    if file_name not in st.session_state.lighthouse_reports:
                        st.session_state.lighthouse_reports[file_name] = report_data
                        st.session_state.report_loaded = True  # ← Flag para notificación
                
                except json.JSONDecodeError:
                    st.error(f"Error al leer {uploaded_file.name}: no es un JSON válido")
        
        # 3. Mostrar reportes cargados
        if "lighthouse_reports" in st.session_state and st.session_state.lighthouse_reports:
            st.success(f"✅ {len(st.session_state.lighthouse_reports)} reporte(s) cargado(s)")
            
            # Listar con opción de eliminar
            for file_name in list(st.session_state.lighthouse_reports.keys()):
                col1, col2 = st.columns([3, 1])
                with col1:
                    st.text(f"📄 {file_name}")
                with col2:
                    if st.button("🗑️", key=f"delete_{file_name}"):
                        del st.session_state.lighthouse_reports[file_name]
                        st.session_state.report_removed = True  # ← Flag para notificación
                        st.rerun()
        else:
            st.info("No hay reportes cargados")

### Session State Variables


Se agregaron nuevas variables al session state de Streamlit:

- **`st.session_state.lighthouse_reports`**: Diccionario {filename: report_data}
- **`st.session_state.report_loaded`**: Flag booleano que indica cuando se cargó un nuevo reporte
- **`st.session_state.report_removed`**: Flag booleano que indica cuando se eliminó un reporte

Estos flags permiten mostrar mensajes de confirmación en el chat.

### `ui/chat.py` - Notificaciones

Se agregó lógica para mostrar mensajes de confirmación cuando se cargan/eliminan reportes.

In [ ]:
import streamlit as st
from core.model import get_model_response


def render_chat():
    """
    Renderiza el chat con notificaciones automáticas de carga/eliminación de reportes.
    """
    
    if "messages" not in st.session_state:
        st.session_state.messages = []
    
    # NUEVO: Detectar carga de reportes
    if st.session_state.get("report_loaded", False):
        num_reports = len(st.session_state.get("lighthouse_reports", {}))
        report_names = list(st.session_state.get("lighthouse_reports", {}).keys())
        
        if num_reports == 1:
            msg = f"📊 Reporte cargado: **{report_names[0]}**. Ahora puedes preguntarme sobre el análisis."
        else:
            msg = f"📊 {num_reports} reportes cargados. Puedo analizar cualquiera de ellos."
        
        # Agregar mensaje al historial (se renderizará después)
        st.session_state.messages.append({"role": "assistant", "content": msg})
        st.session_state.report_loaded = False  # Reset flag
    
    # NUEVO: Detectar eliminación de reportes
    if st.session_state.get("report_removed", False):
        remaining = len(st.session_state.get("lighthouse_reports", {}))
        if remaining > 0:
            msg = f"🗑️ Reporte eliminado. Quedan {remaining} reporte(s) cargado(s)."
        else:
            msg = "🗑️ Reporte eliminado. No hay reportes cargados actualmente."
        
        st.session_state.messages.append({"role": "assistant", "content": msg})
        st.session_state.report_removed = False  # Reset flag
    
    # Mostrar historial de mensajes
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
    
    # Input del usuario
    if prompt := st.chat_input():
        with st.chat_message("user"):
            st.markdown(prompt)
        st.session_state.messages.append({"role": "user", "content": prompt})
        
        with st.chat_message("assistant"):
            with st.spinner("Pensando..."):
                # NUEVO: Pasar reportes al modelo
                lighthouse_reports = st.session_state.get("lighthouse_reports", {})
                response = get_model_response(st.session_state.messages, lighthouse_reports)
                st.markdown(response)
        st.session_state.messages.append({"role": "assistant", "content": response})

## 3. Preprocesamiento de Reportes

### `core/model.py` - Función `preprocess_lighthouse_report()`

Esta función reduce el tamaño de los reportes JSON eliminando datos innecesarios.

In [ ]:
def preprocess_lighthouse_report(report: dict) -> dict:
    """
    Reduce el tamaño del reporte de Lighthouse manteniendo solo información esencial.
    
    MANTIENE:
    - categories.*.score, title, description
    - audits.*.score, title, description, displayValue
    - audits.*.details.summary (si existe)
    - Métricas principales: finalUrl, requestedUrl, timing, configSettings
    
    ELIMINA:
    - audits.*.details.items (arrays grandes)
    - full-page-screenshot
    - traces
    - network-requests completos
    - Strings > 5000 caracteres
    
    Reducción esperada: 20-95% dependiendo del reporte
    """
    processed = {}
    
    # 1. Campos principales
    top_level_fields = [
        "finalUrl", "requestedUrl", "fetchTime",
        "userAgent", "environment", "runtimeError", "timing"
    ]
    for field in top_level_fields:
        if field in report:
            processed[field] = report[field]
    
    # 2. ConfigSettings (solo lo esencial)
    if "configSettings" in report:
        config = report["configSettings"]
        processed["configSettings"] = {
            "emulatedFormFactor": config.get("emulatedFormFactor"),
            "locale": config.get("locale"),
            "onlyCategories": config.get("onlyCategories"),
        }
    
    # 3. Categories (scores y títulos)
    if "categories" in report:
        processed["categories"] = {}
        for category_id, category_data in report["categories"].items():
            processed["categories"][category_id] = {
                "id": category_data.get("id"),
                "title": category_data.get("title"),
                "score": category_data.get("score"),
                "description": category_data.get("description"),
            }
    
    # 4. Audits (sin items completos)
    if "audits" in report:
        processed["audits"] = {}
        for audit_id, audit_data in report["audits"].items():
            processed_audit = {
                "id": audit_data.get("id"),
                "title": audit_data.get("title"),
                "description": audit_data.get("description"),
                "score": audit_data.get("score"),
                "scoreDisplayMode": audit_data.get("scoreDisplayMode"),
                "displayValue": audit_data.get("displayValue"),
                "numericValue": audit_data.get("numericValue"),
                "numericUnit": audit_data.get("numericUnit"),
            }
            
            # Procesar details (solo summary, NO items)
            if "details" in audit_data and isinstance(audit_data["details"], dict):
                details = audit_data["details"]
                processed_details = {}
                
                if "summary" in details:
                    processed_details["summary"] = details["summary"]
                
                if "type" in details:
                    processed_details["type"] = details["type"]
                
                # Contar items en lugar de incluirlos
                if "items" in details and isinstance(details["items"], list):
                    processed_details["itemsCount"] = len(details["items"])
                
                if processed_details:
                    processed_audit["details"] = processed_details
            
            processed["audits"][audit_id] = processed_audit
    
    # 5. Eliminar valores muy largos
    processed = _remove_large_values(processed, max_length=5000)
    
    return processed

### Función auxiliar `_remove_large_values()`

In [ ]:
def _remove_large_values(obj: Any, max_length: int = 5000) -> Any:
    """
    Recursivamente elimina valores cuya representación en string supere max_length.
    
    También elimina campos conocidos que son muy grandes:
    - full-page-screenshot
    - screenshot, screenshots
    - trace, traces
    - network-requests
    - items (ya manejado en preprocess)
    """
    if isinstance(obj, dict):
        result = {}
        for key, value in obj.items():
            # Saltar campos conocidos como grandes
            if key in [
                "full-page-screenshot", "screenshot", "screenshots",
                "trace", "traces", "network-requests", "items"
            ]:
                continue
            
            processed_value = _remove_large_values(value, max_length)
            
            # Reemplazar strings muy largos
            if isinstance(processed_value, str) and len(processed_value) > max_length:
                result[key] = f"[Valor muy largo - {len(processed_value)} caracteres]"
            elif processed_value is not None:
                result[key] = processed_value
        
        return result
    
    elif isinstance(obj, list):
        return [_remove_large_values(item, max_length) for item in obj]
    
    elif isinstance(obj, str):
        if len(obj) > max_length:
            return f"[Texto muy largo - {len(obj)} caracteres]"
        return obj
    
    else:
        return obj

### Ejemplo de Reducción

```
Reporte Original:        2,500,000 caracteres (2.5 MB)
         ↓
Reporte Preprocesado:      250,000 caracteres (250 KB)
         ↓
Reducción:                 90% eliminado
```

**Campos eliminados más grandes:**
- `full-page-screenshot`: ~500-1500 KB (base64 de imágenes)
- `audits.*.details.items`: ~300-800 KB (arrays con cientos de entradas)
- `network-requests`: ~100-200 KB
- `trace`: ~200-500 KB

## 4. Sistema de Resumen con LLM

### `core/model.py` - Función `summarize_preprocessed_report()`

Esta función convierte el JSON preprocesado en un resumen en lenguaje natural usando `llama-3.1-8b-instant`.

In [ ]:
def summarize_preprocessed_report(preprocessed: dict) -> str:
    """
    Resume un reporte preprocesado de Lighthouse en texto conciso.
    
    Pipeline:
    1. Convierte JSON a texto
    2. Divide en chunks de 3000 caracteres
    3. Envía cada chunk a llama-3.1-8b-instant
    4. Fusiona resúmenes en texto < 5000 tokens
    
    Returns:
        str: Resumen en texto para usar como contexto
    """
    try:
        client = Groq(api_key=os.getenv("GROQ_API_KEY"))
        
        # 1. Convertir JSON a texto
        report_text = json.dumps(preprocessed, indent=2, ensure_ascii=False)
        
        # 2. Dividir en chunks de 3000 caracteres
        chunk_size = 3000
        chunks = []
        for i in range(0, len(report_text), chunk_size):
            chunks.append(report_text[i : i + chunk_size])
        
        # 3. Prompt para resumir
        summary_prompt = """Resume este fragmento del reporte de Lighthouse manteniendo solo:
- problemas principales
- métricas clave (performance, SEO, accesibilidad)
- oportunidades de mejora
- puntuaciones relevantes
Máximo 800 tokens."""
        
        # 4. Resumir cada chunk con modelo pequeño
        chunk_summaries = []
        for idx, chunk in enumerate(chunks):
            messages = [
                {"role": "system", "content": summary_prompt},
                {"role": "user", "content": f"Fragmento {idx + 1} de {len(chunks)}:\n\n{chunk}"},
            ]
            
            response = client.chat.completions.create(
                model="llama-3.1-8b-instant",  # ← Modelo pequeño y rápido
                messages=messages,
                temperature=0.3,                # ← Más determinista
                max_tokens=800,                 # ← Límite por chunk
                top_p=1,
                stream=False,
            )
            
            chunk_summaries.append(response.choices[0].message.content)
        
        # 5. Fusionar resúmenes
        if len(chunk_summaries) == 1:
            final_summary = chunk_summaries[0]
        else:
            combined_summaries = "\n\n---\n\n".join(chunk_summaries)
            
            # Si el combinado es muy largo, hacer resumen final
            if len(combined_summaries) > 15000:  # ~5000 tokens
                fusion_prompt = """Fusiona estos resúmenes del reporte de Lighthouse en un solo resumen coherente.
                                Mantén:
                                - Todas las puntuaciones de categorías principales
                                - Problemas críticos identificados
                                - Métricas clave de rendimiento
                                - Oportunidades de mejora más importantes
                                Máximo 1500 tokens."""
                
                messages = [
                    {"role": "system", "content": fusion_prompt},
                    {"role": "user", "content": combined_summaries},
                ]
                
                response = client.chat.completions.create(
                    model="llama-3.1-8b-instant",
                    messages=messages,
                    temperature=0.3,
                    max_tokens=1500,  # ← Límite para fusión final
                    top_p=1,
                    stream=False,
                )
                
                final_summary = response.choices[0].message.content
            else:
                final_summary = combined_summaries
        
        return final_summary
    
    except Exception as e:
        # Fallback: información básica
        categories_info = ""
        if "categories" in preprocessed:
            categories_info = "\n\nCategorías:\n"
            for cat_id, cat_data in preprocessed["categories"].items():
                score = cat_data.get("score", 0) * 100
                title = cat_data.get("title", cat_id)
                categories_info += f"- {title}: {score:.0f}/100\n"
        
        return f"Error al resumir reporte: {str(e)}\n\nInformación básica:{categories_info}"

### Parámetros del Modelo para Resumen

| Parámetro | Valor | Razón |
|-----------|-------|-------|
| `model` | `llama-3.1-8b-instant` | Rápido y económico para tareas de resumen |
| `temperature` | `0.3` | Más determinista, resúmenes consistentes |
| `max_tokens` (chunk) | `800` | Resumen conciso por cada fragmento |
| `max_tokens` (fusión) | `1500` | Resumen final más completo |
| `top_p` | `1` | Sin restricción adicional de muestreo |

## 5. Arquitectura Multi-Modelo

### Modelos Utilizados

El sistema ahora usa **dos modelos diferentes de LLaMA** para optimizar costos y calidad:

#### Modelo 1: llama-3.1-8b-instant (Resumen)

**Características:**
- 8 mil millones de parámetros
- Inferencia ultra-rápida ("instant")
- Económico en uso de tokens

**Uso:**
- Procesamiento intermedio
- Generación de resúmenes de reportes
- Conversión de JSON a texto natural

**Configuración:**
```python
temperature=0.3,    # Determinista
max_tokens=800      # Resúmenes concisos
```

#### Modelo 2: llama-3.3-70b-versatile (Análisis)

**Características:**
- 70 mil millones de parámetros
- Más capaz y versátil
- Mejor comprensión contextual

**Uso:**
- Respuestas finales al usuario
- Análisis complejos
- Recomendaciones expertas

**Configuración:**
```python
temperature=0.7,    # Más creativo
max_tokens=2000     # Respuestas detalladas
```

### Comparación de Modelos

| Aspecto | 8b-instant | 70b-versatile |
|---------|------------|---------------|
| **Tamaño** | 8B parámetros | 70B parámetros |
| **Velocidad** | Ultra-rápido | Moderado |
| **Costo** | Bajo | Alto |
| **Capacidad** | Buena para tareas simples | Excelente para análisis complejos |
| **Uso en app** | Resumen de reportes | Respuestas al usuario |
| **Temperature** | 0.3 (determinista) | 0.7 (creativo) |
| **Max Tokens** | 800-1500 | 2000 |

### Ventajas de la Arquitectura Multi-Modelo

1. **Optimización de Costos**: Usa modelo pequeño para tareas rutinarias
2. **Mejor Rendimiento**: Modelo pequeño procesa más rápido
3. **Mayor Calidad**: Modelo grande se enfoca en respuestas complejas
4. **Escalabilidad**: Cada modelo maneja lo que mejor sabe hacer
5. **Eficiencia de Tokens**: Resumen reduce contexto para modelo grande

## 6. Flujo Completo de Procesamiento

### Pipeline de 3 Etapas

```
┌──────────────────────────────────────────────────────────────┐
│              USUARIO SUBE REPORTE JSON                       │
│           (2.5 MB - Lighthouse completo)                     │
└────────────────────────┬─────────────────────────────────────┘
                         │
                         ↓
        ┌────────────────────────────────────┐
        │   ETAPA 1: PREPROCESAMIENTO       │
        │   preprocess_lighthouse_report()  │
        │                                    │
        │   • Elimina screenshots           │
        │   • Elimina items arrays          │
        │   • Elimina traces                │
        │   • Mantiene scores/títulos       │
        │   • Mantiene summaries            │
        └────────────┬───────────────────────┘
                     │
                     ↓
        Reporte Preprocesado (250 KB)
        Reducción: 90%
                     │
                     ↓
        ┌────────────────────────────────────┐
        │   ETAPA 2: RESUMEN CON LLM        │
        │   summarize_preprocessed_report() │
        │                                    │
        │   • Convierte JSON a texto        │
        │   • Divide en chunks (3000 chars) │
        │   • Envía a llama-3.1-8b-instant  │
        │   • Fusiona resúmenes             │
        └────────────┬───────────────────────┘
                     │
                     ↓
        Resumen en Texto (~5 KB)
        < 5000 tokens
                     │
                     ↓
        ┌────────────────────────────────────┐
        │   ETAPA 3: ANÁLISIS FINAL         │
        │   get_model_response()            │
        │                                    │
        │   • Recibe pregunta del usuario   │
        │   • Incluye resumen en contexto   │
        │   • Envía a llama-3.3-70b         │
        │   • Genera respuesta experta      │
        └────────────┬───────────────────────┘
                     │
                     ↓
        ┌────────────────────────────────────┐
        │   RESPUESTA AL USUARIO             │
        │   Análisis detallado y experto    │
        └────────────────────────────────────┘
```

### Modificación en `get_model_response()`

La función principal ahora integra todo el pipeline:

In [ ]:
def get_model_response(messages: list[dict], lighthouse_reports: dict = None) -> str:
    """
    Obtiene respuesta del modelo principal con reportes resumidos en el contexto.
    
    Args:
        messages: Historial de conversación
        lighthouse_reports: Dict de reportes JSON {filename: report_data}
    
    Returns:
        str: Respuesta del modelo o mensaje de error
    """
    try:
        client = Groq(api_key=os.getenv("GROQ_API_KEY"))
        system_message = {"role": "system", "content": SYSTEM_PROMPT}
        
        # NUEVO: Si hay reportes, preprocesar y resumir
        if lighthouse_reports:
            reports_context = "\n\n## REPORTES LIGHTHOUSE DISPONIBLES\n\n"
            reports_context += (
                "El usuario ha cargado los siguientes reportes de Google Lighthouse. "
                "A continuación se presenta un resumen de cada reporte:\n\n"
            )
            
            for file_name, report_data in lighthouse_reports.items():
                # Etapa 1: Preprocesar
                processed_report = preprocess_lighthouse_report(report_data)
                
                # Etapa 2: Resumir
                summary = summarize_preprocessed_report(processed_report)
                
                # Agregar al contexto
                reports_context += f"### Reporte: {file_name}\n\n"
                reports_context += summary
                reports_context += "\n\n---\n\n"
            
            reports_context += (
                "\nUsa estos resúmenes para responder las preguntas del usuario. "
                "Si el usuario hace una pregunta que requiere análisis de un reporte "
                "y ya tienes reportes cargados, analízalos automáticamente. "
                "Si el usuario pregunta algo que no requiere un reporte específico, "
                "responde normalmente con tus conocimientos sobre optimización web.\n\n"
                "NOTA: Los resúmenes incluyen las métricas clave, problemas principales "
                "y oportunidades de mejora identificadas en los reportes."
            )
            
            # Agregar contexto al system prompt
            enhanced_system_prompt = SYSTEM_PROMPT + "\n\n" + reports_context
            system_message = {"role": "system", "content": enhanced_system_prompt}
        
        all_messages = [system_message] + messages
        
        # Etapa 3: Análisis final con modelo grande
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",  # ← Modelo grande
            messages=all_messages,
            temperature=0.7,                   # ← Más creativo
            max_tokens=2000,                   # ← Respuestas detalladas
            top_p=1,
            stream=False,
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        return f"Lo siento, ha ocurrido un error al procesar tu solicitud: {str(e)}"

## 7. Session State Management

### Variables de Session State

Streamlit usa `st.session_state` para mantener datos entre re-ejecuciones de la app.

In [ ]:
# Variables existentes
st.session_state.messages = []  # Historial de chat
# Estructura: [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]

# NUEVAS variables para reportes
st.session_state.lighthouse_reports = {}  # Reportes cargados
# Estructura: {"reporte1.json": {<datos JSON>}, "reporte2.json": {<datos JSON>}}

st.session_state.report_loaded = False    # Flag: se cargó un nuevo reporte
st.session_state.report_removed = False   # Flag: se eliminó un reporte

### Ciclo de Vida de los Flags

```python
# 1. Usuario sube archivo
st.session_state.lighthouse_reports["reporte.json"] = data
st.session_state.report_loaded = True  # Activar flag

# 2. render_chat() detecta el flag
if st.session_state.get("report_loaded", False):
    # Mostrar mensaje de confirmación
    st.session_state.messages.append({...})
    st.session_state.report_loaded = False  # Reset flag

# 3. Flag se resetea, no se mostrará de nuevo hasta próxima carga
```

## 8. Ejemplos de Uso

### Ejemplo 1: Cargar y Analizar un Reporte

**Paso 1: Usuario sube `mi-sitio.json` via sidebar**

```
Sistema:
  1. Valida JSON ✓
  2. Almacena en session_state.lighthouse_reports
  3. Activa flag report_loaded
  4. Muestra en sidebar: "✅ 1 reporte(s) cargado(s)"
```

**Paso 2: Chat muestra mensaje automático**

```
Assistant: 📊 Reporte cargado: mi-sitio.json. Ahora puedes preguntarme sobre el análisis.
```

**Paso 3: Usuario hace pregunta**

```
User: ¿Cuál es el rendimiento de mi sitio?
```

**Paso 4: Sistema procesa**

```
1. Preprocesa mi-sitio.json (2.5 MB → 250 KB)
2. Resume con llama-3.1-8b-instant (250 KB → 5 KB texto)
3. Envía resumen + pregunta a llama-3.3-70b-versatile
4. Recibe respuesta experta
```

**Paso 5: Respuesta al usuario**

```
Assistant: Basado en el reporte de mi-sitio.json, tu sitio tiene un rendimiento de 65/100.

Problemas principales:
• LCP (Largest Contentful Paint): 3.2s - Necesita optimización
• JavaScript no usado: 250 KiB - Oportunidad de mejora
• Contraste de color: 5 elementos con problemas

Recomendaciones prioritarias:
1. Optimizar imágenes para reducir LCP
2. Eliminar JavaScript no usado
3. Mejorar contraste de colores para accesibilidad
```

### Ejemplo 2: Múltiples Reportes

**Usuario sube 3 reportes:**
- `produccion.json`
- `staging.json`
- `desarrollo.json`

**Chat muestra:**
```
Assistant: 📊 3 reportes cargados. Puedo analizar cualquiera de ellos.
```

**Usuario pregunta:**
```
User: Compara el rendimiento entre producción y staging
```

**Sistema:**
```
1. Preprocesa ambos reportes
2. Resume ambos (2 llamadas a 8b-instant)
3. Envía ambos resúmenes al modelo grande
4. Modelo hace análisis comparativo
```

**Respuesta:**
```
Assistant: Comparación entre producción y staging:

Performance:
• Producción: 65/100
• Staging: 78/100 ✓

Staging está mejor optimizado:
- LCP: 2.1s vs 3.2s en producción
- JavaScript: 150 KiB vs 250 KiB en producción

Recomendación: Aplicar las optimizaciones de staging a producción.
```

### Ejemplo 3: Eliminar Reporte

**Usuario hace clic en 🗑️ junto a `desarrollo.json`**

```
Sistema:
  1. Elimina de session_state.lighthouse_reports
  2. Activa flag report_removed
  3. Actualiza sidebar: "✅ 2 reporte(s) cargado(s)"
```

**Chat muestra:**
```
Assistant: 🗑️ Reporte eliminado. Quedan 2 reporte(s) cargado(s).
```

## Resumen Final

### Mejoras Implementadas

1. ✅ **Sistema de carga de reportes** con validación y gestión visual
2. ✅ **Preprocesamiento inteligente** que reduce 20-95% el tamaño de reportes
3. ✅ **Resumen automático con LLM** que convierte JSON a texto natural
4. ✅ **Arquitectura multi-modelo** optimizada para costo y calidad
5. ✅ **Notificaciones en tiempo real** para mejor UX
6. ✅ **Manejo robusto de errores** con fallbacks

### Impacto en el Usuario

- **Más rápido**: Procesamiento optimizado con modelo pequeño
- **Más económico**: Reduce tokens enviados al modelo principal
- **Más escalable**: Maneja reportes de cualquier tamaño
- **Mejor experiencia**: Feedback visual inmediato
- **Más preciso**: El modelo recibe información pre-procesada y estructurada

### Arquitectura Final

```
Usuario → Sube JSON → Preprocesa → Resume (8b) → Analiza (70b) → Respuesta
  2.5 MB      ↓         250 KB        ↓  5 KB         ↓
          Valida JSON   Elimina    Convierte    Análisis
                        datos      a texto      experto
```